In [2]:
# ============================================================================
# DEPENDENCY INSTALLATION
# Run this cell once to install all required packages for GNN
# ============================================================================

# Core dependencies
%pip install numpy scipy scikit-learn pandas matplotlib seaborn -q

# PyTorch (CPU version - change to GPU version if you have CUDA)
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu -q

# PyTorch Geometric and dependencies
%pip install torch-geometric -q
%pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html -q

print("✅ All dependencies installed successfully!")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ All dependencies installed successfully!


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      Traceback (most recent call last):
        File "c:\Users\Ajayj\miniconda3\envs\transECG\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\Ajayj\miniconda3\envs\transECG\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\Ajayj\miniconda3\envs\transECG\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\Ajayj\AppData\Local\Temp\pip-build-env-jaayk44u\overlay\Lib\site-packages\setuptools\build_meta.py", line 331, in g

In [3]:
import os
import pickle
import numpy as np

# Path to your dataset
DATASET_PATH = r"C:\Users\Ajayj\Documents\ResearchData\PPG_Data\PPG_FieldStudy"

def load_ppg_dataset(dataset_path):
    """
    Load synchronized PPG dataset from all subject .pkl files.
    
    Returns:
        dataset (dict): 
            {
              "S1": {"BVP": np.array, "ACC": np.array, "HR": np.array, "Activity": np.array},
              "S2": {...},
              ...
            }
    """
    dataset = {}
    
    for subj in range(1, 16):  # Subjects S1 ... S15
        subj_id = f"S{subj}"
        subj_folder = os.path.join(dataset_path, subj_id)
        pkl_file = os.path.join(subj_folder, f"{subj_id}.pkl")
        
        if not os.path.exists(pkl_file):
            print(f"⚠️ Missing file: {pkl_file}")
            continue
        
        # Load the .pkl file
        with open(pkl_file, "rb") as f:
            data = pickle.load(f, encoding="latin1")  # latin1 ensures compatibility
        
        # Extract signals + labels
        bvp = np.array(data["signal"]["wrist"]["BVP"])   # 64 Hz PPG
        acc = np.array(data["signal"]["wrist"]["ACC"])   # 32 Hz ACC (3D)
        hr  = np.array(data["label"])                    # ECG-derived HR (windowed)
        act = np.array(data["activity"])                 # Activity ID
        
        dataset[subj_id] = {
            "BVP": bvp,
            "ACC": acc,
            "HR": hr,
            "Activity": act
        }
        
        print(f"✅ Loaded {subj_id}: BVP {bvp.shape}, ACC {acc.shape}, HR {hr.shape}, Activity {act.shape}")
    
    return dataset

# Run loader
dataset = load_ppg_dataset(DATASET_PATH)

# Example: check S1 keys
print("\nKeys for S1:", dataset["S1"].keys())


✅ Loaded S1: BVP (589568, 1), ACC (294784, 3), HR (4603,), Activity (36848, 1)
✅ Loaded S2: BVP (525120, 1), ACC (262560, 3), HR (4099,), Activity (32820, 1)
✅ Loaded S3: BVP (559424, 1), ACC (279712, 3), HR (4367,), Activity (34964, 1)
✅ Loaded S4: BVP (585600, 1), ACC (292800, 3), HR (4572,), Activity (36600, 1)
✅ Loaded S5: BVP (595520, 1), ACC (297760, 3), HR (4649,), Activity (37220, 1)
✅ Loaded S6: BVP (336000, 1), ACC (168000, 3), HR (2622,), Activity (21000, 1)
✅ Loaded S7: BVP (597952, 1), ACC (298976, 3), HR (4668,), Activity (37372, 1)
✅ Loaded S8: BVP (517120, 1), ACC (258560, 3), HR (4037,), Activity (32320, 1)
✅ Loaded S9: BVP (547840, 1), ACC (273920, 3), HR (4277,), Activity (34240, 1)
✅ Loaded S10: BVP (681472, 1), ACC (340736, 3), HR (5321,), Activity (42592, 1)
✅ Loaded S11: BVP (579072, 1), ACC (289536, 3), HR (4521,), Activity (36192, 1)
✅ Loaded S12: BVP (506496, 1), ACC (253248, 3), HR (3954,), Activity (31656, 1)
✅ Loaded S13: BVP (584704, 1), ACC (292352, 3), H

In [4]:
import numpy as np

def segment_signal(signal, window_size, step_size):
    """
    Segment 1D or 2D signal into overlapping windows.
    
    Args:
        signal (np.array): shape (N,) or (N, D)
        window_size (int): number of samples per window
        step_size (int): number of samples to move the window
    
    Returns:
        windows (np.array): shape (#windows, window_size, D)
    """
    n_samples = signal.shape[0]
    if signal.ndim == 1:
        signal = signal.reshape(-1, 1)
    n_dims = signal.shape[1]
    
    windows = []
    for start in range(0, n_samples - window_size + 1, step_size):
        end = start + window_size
        windows.append(signal[start:end])
    return np.array(windows)

def preprocess_dataset(dataset):
    """
    For each subject, segment BVP and ACC signals into 8s windows
    with 2s shift and align them with HR labels.
    
    Returns:
        processed (dict): {subject: {"BVP": ..., "ACC": ..., "HR": ..., "Activity": ...}}
    """
    processed = {}
    
    # Parameters
    bvp_fs = 64   # Hz
    acc_fs = 32   # Hz
    win_time = 8  # seconds
    step_time = 2 # seconds
    
    bvp_win = win_time * bvp_fs   # 512 samples
    bvp_step = step_time * bvp_fs # 128 samples
    
    acc_win = win_time * acc_fs   # 256 samples
    acc_step = step_time * acc_fs # 64 samples
    
    for subj, data in dataset.items():
        bvp = data["BVP"].squeeze()    # (N,)
        acc = data["ACC"]              # (N, 3)
        hr  = data["HR"]               # (#windows,)
        act = data["Activity"]         # (#activity_samples, 1) at 4 Hz
        
        # Segment BVP + ACC
        bvp_windows = segment_signal(bvp, bvp_win, bvp_step)
        acc_windows = segment_signal(acc, acc_win, acc_step)
        
        # Align to HR length
        min_len = min(len(hr), len(bvp_windows), len(acc_windows))
        bvp_windows = bvp_windows[:min_len]
        acc_windows = acc_windows[:min_len]
        hr = hr[:min_len]
        
        processed[subj] = {
            "BVP": bvp_windows,   # (N_windows, 512, 1)
            "ACC": acc_windows,   # (N_windows, 256, 3)
            "HR": hr,             # (N_windows,)
            "Activity": act       # Keep raw activity for now
        }
        
        print(f"✅ {subj}: {bvp_windows.shape}, {acc_windows.shape}, HR {hr.shape}")
    
    return processed

# Run preprocessing
processed_dataset = preprocess_dataset(dataset)

# Example check
print("\nS1 processed shapes:")
print("BVP:", processed_dataset["S1"]["BVP"].shape)
print("ACC:", processed_dataset["S1"]["ACC"].shape)
print("HR:", processed_dataset["S1"]["HR"].shape)


✅ S1: (4603, 512, 1), (4603, 256, 3), HR (4603,)
✅ S2: (4099, 512, 1), (4099, 256, 3), HR (4099,)
✅ S3: (4367, 512, 1), (4367, 256, 3), HR (4367,)
✅ S4: (4572, 512, 1), (4572, 256, 3), HR (4572,)
✅ S5: (4649, 512, 1), (4649, 256, 3), HR (4649,)
✅ S6: (2622, 512, 1), (2622, 256, 3), HR (2622,)
✅ S7: (4668, 512, 1), (4668, 256, 3), HR (4668,)
✅ S8: (4037, 512, 1), (4037, 256, 3), HR (4037,)
✅ S9: (4277, 512, 1), (4277, 256, 3), HR (4277,)
✅ S10: (5321, 512, 1), (5321, 256, 3), HR (5321,)
✅ S11: (4521, 512, 1), (4521, 256, 3), HR (4521,)
✅ S12: (3954, 512, 1), (3954, 256, 3), HR (3954,)
✅ S13: (4565, 512, 1), (4565, 256, 3), HR (4565,)
✅ S14: (4476, 512, 1), (4476, 256, 3), HR (4476,)
✅ S15: (3966, 512, 1), (3966, 256, 3), HR (3966,)

S1 processed shapes:
BVP: (4603, 512, 1)
ACC: (4603, 256, 3)
HR: (4603,)


In [5]:
import numpy as np
import scipy.stats as stats
from scipy.signal import welch

def extract_bvp_features(window, fs=64):
    """Extract time + frequency features from one BVP window (512 samples)."""
    feats = []
    
    # Time-domain
    feats.append(np.mean(window))
    feats.append(np.std(window))
    feats.append(np.min(window))
    feats.append(np.max(window))
    feats.append(stats.skew(window))
    feats.append(stats.kurtosis(window))
    
    # Frequency-domain (Welch PSD)
    freqs, psd = welch(window, fs=fs, nperseg=256)
    
    # Bands: low freq (0.04-0.15 Hz), high freq (0.15-0.4 Hz)
    lf_band = np.logical_and(freqs >= 0.04, freqs < 0.15)
    hf_band = np.logical_and(freqs >= 0.15, freqs < 0.4)
    
    lf_power = np.sum(psd[lf_band])
    hf_power = np.sum(psd[hf_band])
    
    feats.append(lf_power)
    feats.append(hf_power)
    
    # Dominant frequency
    dom_idx = np.argmax(psd)
    feats.append(freqs[dom_idx])
    feats.append(psd[dom_idx])
    
    return np.array(feats)

def safe_corrcoef(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return np.corrcoef(x, y)[0,1]

def extract_acc_features(window, fs=32):
    """Extract motion features from one ACC window (256×3)."""
    feats = []
    for i in range(3):  # x, y, z axes
        axis = window[:, i]
        feats.append(np.mean(axis))
        feats.append(np.std(axis))
        feats.append(np.min(axis))
        feats.append(np.max(axis))
        feats.append(np.sum(axis**2))  # energy
    
    # Cross-axis correlations
    feats.append(safe_corrcoef(window[:,0], window[:,1]))
    feats.append(safe_corrcoef(window[:,0], window[:,2]))
    feats.append(safe_corrcoef(window[:,1], window[:,2]))
    
    return np.array(feats)

def build_feature_dataset(processed_dataset):
    """
    Extract features from BVP + ACC for all subjects.
    Returns:
        X (np.array): features [N_windows, N_features]
        y (np.array): HR labels [N_windows]
        subjects (list): subject IDs per window
    """
    X, y, subjects = [], [], []
    
    for subj, data in processed_dataset.items():
        bvp_windows = data["BVP"]
        acc_windows = data["ACC"]
        hr_labels = data["HR"]
        
        for i in range(len(hr_labels)):
            bvp_feats = extract_bvp_features(bvp_windows[i].squeeze())
            acc_feats = extract_acc_features(acc_windows[i])
            feats = np.concatenate([bvp_feats, acc_feats])
            
            X.append(feats)
            y.append(hr_labels[i])
            subjects.append(subj)
    
    return np.array(X), np.array(y), subjects

# Run feature extraction
X, y, subj_ids = build_feature_dataset(processed_dataset)

print("Feature dataset shape:", X.shape)
print("Labels shape:", y.shape)
print("Example feature vector length:", X.shape[1])


Feature dataset shape: (64697, 28)
Labels shape: (64697,)
Example feature vector length: 28


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def prepare_train_test(X, y, test_size=0.2, random_state=42):
    """
    Normalize features and split into train/test sets.
    """
    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, random_state=random_state
    )
    
    print("Train set:", X_train.shape, "Test set:", X_test.shape)
    return X_train, X_test, y_train, y_test, scaler

# Run preparation
X_train, X_test, y_train, y_test, scaler = prepare_train_test(X, y)

print("✅ Dataset ready for modeling")


Train set: (51757, 28) Test set: (12940, 28)
✅ Dataset ready for modeling


In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from sklearn.neighbors import kneighbors_graph
import numpy as np

def build_knn_graph(X_train, X_test, y_train, y_test, k=10):
    """
    Build k-NN graph from feature vectors for GNN training.
    
    Args:
        X_train: Training features (N_train, num_features)
        X_test: Test features (N_test, num_features)
        y_train: Training labels (HR values)
        y_test: Test labels (HR values)
        k: Number of nearest neighbors
    
    Returns:
        train_data: PyTorch Geometric Data object for training
        test_data: PyTorch Geometric Data object for testing
    """
    print(f"Building k-NN graph with k={k}...")
    
    # Build train graph
    # Create k-NN adjacency matrix (mode='connectivity' gives binary adjacency)
    train_adj = kneighbors_graph(X_train, k, mode='connectivity', include_self=False)
    train_edge_index = torch.tensor(np.array(train_adj.nonzero()), dtype=torch.long)
    
    # Convert to PyG Data object
    train_data = Data(
        x=torch.tensor(X_train, dtype=torch.float32),
        edge_index=train_edge_index,
        y=torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
    )
    
    # Build test graph
    test_adj = kneighbors_graph(X_test, k, mode='connectivity', include_self=False)
    test_edge_index = torch.tensor(np.array(test_adj.nonzero()), dtype=torch.long)
    
    test_data = Data(
        x=torch.tensor(X_test, dtype=torch.float32),
        edge_index=test_edge_index,
        y=torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
    )
    
    print(f"✅ Train graph: {train_data.num_nodes} nodes, {train_data.num_edges} edges")
    print(f"✅ Test graph: {test_data.num_nodes} nodes, {test_data.num_edges} edges")
    
    return train_data, test_data

# Build k-NN graphs for GNN
train_data, test_data = build_knn_graph(X_train, X_test, y_train, y_test, k=10)

print(f"\nTrain data shape: {train_data.x.shape}")
print(f"Test data shape: {test_data.x.shape}")

c:\Users\Ajayj\miniconda3\envs\transECG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Building k-NN graph with k=10...
✅ Train graph: 51757 nodes, 517570 edges
✅ Test graph: 12940 nodes, 129400 edges

Train data shape: torch.Size([51757, 28])
Test data shape: torch.Size([12940, 28])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import os
from datetime import datetime

class GAT_HR_Regressor(nn.Module):
    """
    Graph Attention Network (GAT) for Heart Rate Regression.
    
    Architecture:
    - 2 GAT layers with multi-head attention
    - Batch normalization and dropout for regularization
    - Final MLP for regression
    """
    def __init__(self, in_channels, hidden_channels=64, heads=4, dropout=0.3):
        super(GAT_HR_Regressor, self).__init__()
        
        # GAT Layer 1: multi-head attention
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.bn1 = nn.BatchNorm1d(hidden_channels * heads)
        
        # GAT Layer 2: multi-head attention
        self.conv2 = GATConv(hidden_channels * heads, hidden_channels, heads=heads, dropout=dropout)
        self.bn2 = nn.BatchNorm1d(hidden_channels * heads)
        
        # Regression head (MLP)
        self.fc1 = nn.Linear(hidden_channels * heads, hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, 1)
        
        self.dropout = dropout
        
    def forward(self, x, edge_index):
        # GAT Layer 1
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # GAT Layer 2
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Regression head
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        
        return x

def train_gnn_epoch(model, data, optimizer, device):
    """Train GNN for one epoch."""
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    out = model(data.x.to(device), data.edge_index.to(device))
    loss = F.mse_loss(out, data.y.to(device))
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    return loss.item()

def evaluate_gnn(model, data, device):
    """Evaluate GNN on a dataset."""
    model.eval()
    with torch.no_grad():
        out = model(data.x.to(device), data.edge_index.to(device))
        y_pred = out.cpu().numpy().squeeze()
        y_true = data.y.cpu().numpy().squeeze()
        
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        
    return mae, rmse, r2, y_pred

def train_gnn(train_data, test_data, epochs=500, lr=0.001, hidden_channels=64, heads=4, dropout=0.3, 
              save_checkpoints=True, checkpoint_freq=50):
    """
    Train a GAT-based GNN for heart rate prediction with AUTO-SAVING checkpoints.
    
    Args:
        save_checkpoints: If True, automatically saves model checkpoints during training
        checkpoint_freq: Save checkpoint every N epochs (default: 50)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Create checkpoint directory
    if save_checkpoints:
        os.makedirs('saved_models/checkpoints', exist_ok=True)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        checkpoint_dir = f'saved_models/checkpoints/training_{timestamp}'
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"📁 Checkpoints will be saved to: {checkpoint_dir}")
    
    # Initialize model
    in_channels = train_data.x.shape[1]
    model = GAT_HR_Regressor(in_channels, hidden_channels, heads, dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    
    print(f"\n🚀 Training GAT-GNN for {epochs} epochs...")
    print(f"Model parameters: hidden={hidden_channels}, heads={heads}, dropout={dropout}")
    if save_checkpoints:
        print(f"💾 Auto-save enabled: checkpoint every {checkpoint_freq} epochs\n")
    
    # Training loop
    best_test_mae = float('inf')
    best_model_state = None
    
    for epoch in range(1, epochs + 1):
        loss = train_gnn_epoch(model, train_data, optimizer, device)
        
        if epoch % 25 == 0 or epoch == 1:
            train_mae, train_rmse, train_r2, _ = evaluate_gnn(model, train_data, device)
            test_mae, test_rmse, test_r2, _ = evaluate_gnn(model, test_data, device)
            
            print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | "
                  f"Train MAE: {train_mae:.2f} | Test MAE: {test_mae:.2f} | "
                  f"Test R²: {test_r2:.3f}")
            
            # Save best model
            if test_mae < best_test_mae:
                best_test_mae = test_mae
                best_model_state = model.state_dict().copy()
                
                # Save best model checkpoint
                if save_checkpoints:
                    best_model_path = f'{checkpoint_dir}/best_model.pt'
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': best_model_state,
                        'optimizer_state_dict': optimizer.state_dict(),
                        'test_mae': test_mae,
                        'test_rmse': test_rmse,
                        'test_r2': test_r2,
                        'model_config': {
                            'in_channels': in_channels,
                            'hidden_channels': hidden_channels,
                            'heads': heads,
                            'dropout': dropout
                        },
                        'training_config': {'lr': lr, 'epochs': epochs, 'k_neighbors': 10},
                        'scaler': scaler
                    }, best_model_path)
        
        # Periodic checkpoint save
        if save_checkpoints and epoch % checkpoint_freq == 0:
            checkpoint_path = f'{checkpoint_dir}/checkpoint_epoch_{epoch}.pt'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_test_mae': best_test_mae,
                'model_config': {
                    'in_channels': in_channels,
                    'hidden_channels': hidden_channels,
                    'heads': heads,
                    'dropout': dropout
                },
                'training_config': {'lr': lr, 'epochs': epochs, 'k_neighbors': 10},
                'scaler': scaler
            }, checkpoint_path)
            print(f"💾 Checkpoint saved: epoch {epoch}")
    
    # Load best model
    model.load_state_dict(best_model_state)
    
    # Save final model
    if save_checkpoints:
        final_model_path = f'{checkpoint_dir}/final_model_complete.pt'
        torch.save({
            'model_state_dict': best_model_state,
            'model_config': {
                'in_channels': in_channels,
                'hidden_channels': hidden_channels,
                'heads': heads,
                'dropout': dropout
            },
            'training_config': {'lr': lr, 'epochs': epochs, 'k_neighbors': 10},
            'scaler': scaler,
            'feature_info': {
                'num_features': X_train.shape[1],
                'num_train_samples': len(X_train),
                'num_test_samples': len(X_test)
            },
            'timestamp': timestamp
        }, final_model_path)
        print(f"\n💾 Final model saved to: {final_model_path}")
    
    # Final evaluation
    print("\n" + "="*70)
    print("✅ Training Complete - Final Test Performance")
    print("="*70)
    test_mae, test_rmse, test_r2, y_pred = evaluate_gnn(model, test_data, device)
    print(f"MAE:  {test_mae:.2f} bpm")
    print(f"RMSE: {test_rmse:.2f} bpm")
    print(f"R²:   {test_r2:.3f}")
    print("="*70)
    
    if save_checkpoints:
        print(f"\n📦 All files saved in: {checkpoint_dir}")
        print("   - best_model.pt (updated whenever performance improves)")
        print("   - checkpoint_epoch_*.pt (periodic saves)")
        print("   - final_model_complete.pt (ready to share with your group)")
    
    return model, y_pred

# Train the GNN model with AUTO-SAVING enabled
# ⚠️ IMPORTANT: Checkpoints are saved automatically every 50 epochs!
gnn_model, y_pred_gnn = train_gnn(
    train_data, 
    test_data, 
    epochs=500,
    lr=0.001,
    hidden_channels=64,
    heads=4,
    dropout=0.3,
    save_checkpoints=True,  # AUTO-SAVE ENABLED!
    checkpoint_freq=50       # Save every 50 epochs
)

In [ ]:
import itertools
import torch

def tune_gnn_hyperparameters(train_data, test_data, epochs=250):
    """
    Grid search hyperparameter tuning for GAT-GNN.
    
    Tests different combinations of:
    - hidden_channels: model capacity
    - heads: number of attention heads
    - dropout: regularization strength
    - learning_rate: optimization step size
    """
    
    param_grid = {
        'hidden_channels': [32, 64, 128],
        'heads': [2, 4, 8],
        'dropout': [0.2, 0.3, 0.5],
        'lr': [0.001, 0.0005]
    }
    
    print("🔍 Starting GNN Hyperparameter Tuning...")
    print(f"Testing {np.prod([len(v) for v in param_grid.values()])} combinations\n")
    
    best_mae = float('inf')
    best_params = None
    results = []
    
    # Grid search
    for hidden_channels, heads, dropout, lr in itertools.product(
        param_grid['hidden_channels'],
        param_grid['heads'],
        param_grid['dropout'],
        param_grid['lr']
    ):
        print(f"Testing: hidden={hidden_channels}, heads={heads}, dropout={dropout}, lr={lr}")
        
        try:
            # Train model with current hyperparameters
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            in_channels = train_data.x.shape[1]
            
            model = GAT_HR_Regressor(in_channels, hidden_channels, heads, dropout).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
            
            # Train for specified epochs
            for epoch in range(epochs):
                train_gnn_epoch(model, train_data, optimizer, device)
            
            # Evaluate on test set
            test_mae, test_rmse, test_r2, _ = evaluate_gnn(model, test_data, device)
            
            results.append({
                'hidden_channels': hidden_channels,
                'heads': heads,
                'dropout': dropout,
                'lr': lr,
                'test_mae': test_mae,
                'test_rmse': test_rmse,
                'test_r2': test_r2
            })
            
            print(f"  → MAE: {test_mae:.2f}, RMSE: {test_rmse:.2f}, R²: {test_r2:.3f}\n")
            
            # Track best model
            if test_mae < best_mae:
                best_mae = test_mae
                best_params = {
                    'hidden_channels': hidden_channels,
                    'heads': heads,
                    'dropout': dropout,
                    'lr': lr
                }
                
        except Exception as e:
            print(f"  ⚠️ Failed with error: {e}\n")
            continue
    
    print("\n" + "="*70)
    print("✅ Best GNN Hyperparameters Found:")
    print("="*70)
    for key, value in best_params.items():
        print(f"{key}: {value}")
    print(f"\nBest Test MAE: {best_mae:.2f} bpm")
    print("="*70)
    
    return best_params, results

# Run hyperparameter tuning (⚠️ This will take some time)
# Comment out if you want to skip tuning
best_params, tuning_results = tune_gnn_hyperparameters(train_data, test_data, epochs=250)

# Train final model with best hyperparameters
print("\n🚀 Training final GNN model with best hyperparameters...")
best_gnn_model, y_pred_best_gnn = train_gnn(
    train_data,
    test_data,
    epochs=500,  # Increased from 200 to 500 for longer training
    lr=best_params['lr'],
    hidden_channels=best_params['hidden_channels'],
    heads=best_params['heads'],
    dropout=best_params['dropout']
)

In [ ]:
# Visualize GNN predictions
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def evaluate_model(y_test, y_pred, sample_range=500, title_prefix="GNN"):
    # 1. Scatter Plot
    plt.figure(figsize=(6,6))
    sns.scatterplot(x=y_test, y=y_pred, alpha=0.4)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel("Ground Truth HR (bpm)")
    plt.ylabel("Predicted HR (bpm)")
    plt.title(f"{title_prefix} - Predicted vs Ground Truth HR")
    plt.show()
    
    # 2. Time Series (first N samples)
    plt.figure(figsize=(12,4))
    plt.plot(y_test[:sample_range], label="Ground Truth", lw=2)
    plt.plot(y_pred[:sample_range], label="Predicted", lw=2, alpha=0.7)
    plt.xlabel("Sample Index")
    plt.ylabel("Heart Rate (bpm)")
    plt.title(f"{title_prefix} - Prediction vs Ground Truth (first {sample_range} samples)")
    plt.legend()
    plt.show()
    
    # 3. Error Distribution
    errors = y_pred - y_test
    plt.figure(figsize=(6,4))
    sns.histplot(errors, bins=50, kde=True)
    plt.xlabel("Prediction Error (bpm)")
    plt.title(f"{title_prefix} - Error Distribution")
    plt.show()
    
    # 4. Bland-Altman Plot
    mean_hr = (y_test + y_pred) / 2
    diff_hr = y_pred - y_test
    plt.figure(figsize=(6,4))
    plt.scatter(mean_hr, diff_hr, alpha=0.4)
    plt.axhline(np.mean(diff_hr), color='red', linestyle='--', label=f'Mean: {np.mean(diff_hr):.2f}')
    plt.axhline(np.mean(diff_hr) + 1.96*np.std(diff_hr), color='gray', linestyle='--', label=f'+1.96 SD: {np.mean(diff_hr) + 1.96*np.std(diff_hr):.2f}')
    plt.axhline(np.mean(diff_hr) - 1.96*np.std(diff_hr), color='gray', linestyle='--', label=f'-1.96 SD: {np.mean(diff_hr) - 1.96*np.std(diff_hr):.2f}')
    plt.xlabel("Mean HR (bpm)")
    plt.ylabel("Difference (Pred - True, bpm)")
    plt.title(f"{title_prefix} - Bland-Altman Plot")
    plt.legend()
    plt.show()

# Run evaluation on GNN predictions
evaluate_model(y_test, y_pred_gnn, sample_range=500, title_prefix="GAT-GNN")

# If you ran hyperparameter tuning, evaluate the best model
# evaluate_model(y_test, y_pred_best_gnn, sample_range=500, title_prefix="Best GAT-GNN")

In [ ]:
# ============================================================================
# SAVE TRAINED GNN MODEL
# Run this cell to save your trained model for sharing with your group
# ============================================================================

import torch
import pickle
import os
from datetime import datetime

# Create a models directory if it doesn't exist
os.makedirs('saved_models', exist_ok=True)

# Get timestamp for versioning
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Determine which model to save
# If you ran hyperparameter tuning, save best_gnn_model, otherwise save gnn_model
try:
    model_to_save = best_gnn_model
    model_name = "best_gnn_model"
    config = best_params  # Use the best hyperparameters found during tuning
    print("📦 Saving hyperparameter-tuned model...")
except NameError:
    model_to_save = gnn_model
    model_name = "gnn_model"
    config = {
        'hidden_channels': 64,
        'heads': 4,
        'dropout': 0.3,
        'lr': 0.001
    }
    print("📦 Saving base GNN model...")

# Get model architecture info
in_channels = train_data.x.shape[1]

# Save complete model package
model_package = {
    'model_state_dict': model_to_save.state_dict(),
    'model_config': {
        'in_channels': in_channels,
        'hidden_channels': config['hidden_channels'],
        'heads': config['heads'],
        'dropout': config['dropout']
    },
    'training_config': {
        'lr': config.get('lr', 0.001),
        'epochs': 500,
        'k_neighbors': 10
    },
    'scaler': scaler,  # Save the StandardScaler for feature normalization
    'feature_info': {
        'num_features': X_train.shape[1],
        'num_train_samples': len(X_train),
        'num_test_samples': len(X_test)
    },
    'timestamp': timestamp
}

# Save the complete package
model_path = f'saved_models/gat_hr_model_complete_{timestamp}.pt'
torch.save(model_package, model_path)
print(f"✅ Complete model package saved to: {model_path}")

# Also save a lightweight version (just weights) for easier sharing
weights_path = f'saved_models/gat_hr_model_weights_{timestamp}.pt'
torch.save(model_to_save.state_dict(), weights_path)
print(f"✅ Model weights saved to: {weights_path}")

# Save model configuration as a readable text file
config_path = f'saved_models/model_config_{timestamp}.txt'
with open(config_path, 'w') as f:
    f.write("="*70 + "\n")
    f.write("GAT-GNN Heart Rate Prediction Model Configuration\n")
    f.write("="*70 + "\n\n")
    f.write(f"Saved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write("Model Architecture:\n")
    f.write(f"  - Input features: {in_channels}\n")
    f.write(f"  - Hidden channels: {config['hidden_channels']}\n")
    f.write(f"  - Attention heads: {config['heads']}\n")
    f.write(f"  - Dropout: {config['dropout']}\n\n")
    f.write("Training Configuration:\n")
    f.write(f"  - Learning rate: {config.get('lr', 0.001)}\n")
    f.write(f"  - Epochs: 500\n")
    f.write(f"  - K-neighbors: 10\n\n")
    f.write("Dataset Info:\n")
    f.write(f"  - Training samples: {len(X_train)}\n")
    f.write(f"  - Test samples: {len(X_test)}\n")
    f.write(f"  - Feature dimensions: {X_train.shape[1]}\n")
    f.write("="*70 + "\n")

print(f"✅ Configuration saved to: {config_path}")

print("\n" + "="*70)
print("📁 FILES TO SHARE WITH YOUR GROUP:")
print("="*70)
print(f"1. {model_path}")
print(f"   → Complete model package (includes everything)")
print(f"\n2. {weights_path}")
print(f"   → Just the model weights (lightweight)")
print(f"\n3. {config_path}")
print(f"   → Human-readable configuration")
print("\n4. saved_models/model_loader.py")
print("   → Script to load and use the model (will be created next)")
print("="*70)

# Evaluate final model performance for documentation
print("\n" + "="*70)
print("FINAL MODEL PERFORMANCE:")
print("="*70)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_mae, test_rmse, test_r2, _ = evaluate_gnn(model_to_save, test_data, device)
print(f"Test MAE:  {test_mae:.2f} bpm")
print(f"Test RMSE: {test_rmse:.2f} bpm")
print(f"Test R²:   {test_r2:.3f}")
print("="*70)